In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ai agent trainning/train/sft')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
import yaml

with open('../configs/base.yaml', 'r') as file:
    configs = yaml.safe_load(file)

with open('../configs/sft.yaml', 'r') as file:
    configs.update(yaml.safe_load(file))

# Inference

In [ ]:
max_new_tokens = 64

messages = [
    {"role": "system", "content": "Bạn là một trợ lý hữu ích trong lĩnh vực chăm sóc sức khoẻ,\
    hãy phản hồi từng bước, đầu tiên là xin chào, sau đó là phân tích về câu hỏi, rồi phản hồi lại dựa trên kiến thức vừa tìm hiểu, đánh giá câu hỏi của user có liên quan đến sức khoẻ hay không. \
    những câu hỏi không liên quan đến sức khoẻ như xe cộ, thể thao, hàng không, vũ trụ, du lịch,... hãy trả lời là 'không biết'. \
    hãy suy nghĩ từng bước một, step by step."},

    {"role": "user", "content": "đau đầu là do nguyên nhân nào"},
]

model = ""
unsloth_model, unsloth_tokenizer = None, None
peft_model, peft_tokenizer = None, None
import gc

## Unsloth

In [ ]:
from transformers import TextStreamer

In [ ]:
if model != "unsloth":
    print("unsloth loading...")
    del peft_model, peft_tokenizer
    gc.collect()
    model = "unsloth"

    from unsloth import FastLanguageModel
    unsloth_model, unsloth_tokenizer = FastLanguageModel.from_pretrained(
        model_name = configs['pretrain_model'], # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = configs['model']['max_seq_length'],
        dtype = configs['model']['dtype'],
        load_in_4bit = configs['model']['load_in_4bit'],
    )
    FastLanguageModel.for_inference(unsloth_model) # Enable native 2x faster inference

unsloth loading...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


<ipython-input-10-266ad7892883>:7: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.4.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
if model == "unsloth":
    inputs = unsloth_tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = unsloth_model.generate(input_ids = inputs, max_new_tokens = max_new_tokens, use_cache = True,
                            temperature = 1.5, min_p = 0.1)

    print(unsloth_tokenizer.batch_decode(outputs))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nBạn là một trợ lý hữu ích trong lĩnh vực chăm sóc sức khoẻ,    hãy phản hồi từng bước, đầu tiên là xin chào, sau đó là phân tích về câu hỏi, rồi phản hồi lại dựa trên kiến thức vừa tìm hiểu, đánh giá câu hỏi của user có liên quan đến sức khoẻ hay không.     những câu hỏi không liên quan đến sức khoẻ như xe cộ, thể thao, hàng không, vũ trụ, du lịch,... hãy trả lời là 'không biết'.     hãy suy nghĩ từng bước một, step by step.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nđau đầu là do nguyên nhân nào<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nCảm ơn câu hỏi của bạn. Tôi hiểu mối quan tâm của bạn. Da đầu có thể bị đau do rất nhiều nguyên nhân khác nhau. Vì vậy, tôi khuyên bạn nên kiểm tra các yếu tố bên ngoài. Ví dụ như loại sữa đánh răng của bạn là gì. Bạn thường sử dụng máy"]


In [ ]:
if model == "unsloth":
    inputs = unsloth_tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    text_streamer = TextStreamer(unsloth_tokenizer, skip_prompt = True)
    _ = unsloth_model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = max_new_tokens,
                      use_cache = True, temperature = 1.5, min_p = 0.1)

Cảm ơn câu hỏi của bạn. Đau đầu là nguyên nhân do lo lắng, căng thẳng, căng thẳng về tinh thần và cả về thể chất. Nguyên nhân đau đầu là do nhiều tình huống. Trong đó:1. Đau đầu nặng có thể được gây ra do rối lo


## Transformers

In [ ]:
import torch

In [ ]:
if model != "peft":
    print("peft loading...")
    del unsloth_model, unsloth_tokenizer
    gc.collect()
    model = "peft"

    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer, AutoModelForCausalLM

    peft_model = AutoPeftModelForCausalLM.from_pretrained(
        configs['pretrain_model'], # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = configs['model']['load_in_4bit'],
    )
    peft_tokenizer = AutoTokenizer.from_pretrained(configs['pretrain_model'])

peft loading...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model wil

In [ ]:
if model == "peft": # đã chạy unsloth thì chạy dòng này sẽ lỗi
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    peft_model.to(device)
    peft_model.eval()

    inputs = peft_tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to(device)

    with torch.no_grad():
        outputs = peft_model.generate(input_ids = inputs, max_new_tokens = max_new_tokens, use_cache = True,
                          temperature = 1.5, min_p = 0.1)
        print(peft_tokenizer.decode(outputs[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Bạn là một trợ lý hữu ích trong lĩnh vực chăm sóc sức khoẻ,    hãy phản hồi từng bước, đầu tiên là xin chào, sau đó là phân tích về câu hỏi, rồi phản hồi lại dựa trên kiến thức vừa tìm hiểu, đánh giá câu hỏi của user có liên quan đến sức khoẻ hay không.     những câu hỏi không liên quan đến sức khoẻ như xe cộ, thể thao, hàng không, vũ trụ, du lịch,... hãy trả lời là 'không biết'.     hãy suy nghĩ từng bước một, step by step.<|eot_id|><|start_header_id|>user<|end_header_id|>

đau đầu là do nguyên nhân nào<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Xin chào bạn có vẻ đau đầu. Dấu hiệu này thường gây sốt và đau cơ. Do vậy tôi nghĩ có thể nó là do nhiễm trùng, cảm cúm, lyme, hoặc bất kỳ bệnh tự do nào trong tương lai. Đừng lo lắng về điều này. Bạn sẽ ổn
